# Pokemon Match

This file calculates the top x number of attacking pokemon per type for raids (PvE). It lists:
- the top moves for each type
- a summary of the pokemon for each type (split by normal/shadow) 
- a combined summary of all pokemon of all types at the end (split norm/shadow)

Use the "Runtime -> Run All" to see all output :)

In [ ]:
import os
import shutil
print(os.getcwd())

/content


In [ ]:
# this bit imports the github, clones and changes the path:
os.chdir('/content')
myDir = './drmckenzie.github.io/'

# WARNING! this will delete any changes to the files you have made
if os.path.isdir(myDir):
    shutil.rmtree(myDir)

!git clone https://github.com/drmckenzie/drmckenzie.github.io.git

os.chdir(myDir)
print(os.getcwd())

Cloning into 'drmckenzie.github.io'...
remote: Enumerating objects: 164, done.
remote: Counting objects: 100% (164/164), done.
remote: Compressing objects: 100% (99/99), done.
remote: Total 164 (delta 99), reused 118 (delta 62), pack-reused 0
Receiving objects: 100% (164/164), 1.11 MiB | 6.00 MiB/s, done.
Resolving deltas: 100% (99/99), done.
/content/drmckenzie.github.io


# You can change these default settings to customise the results:

In [ ]:
# This is the list of types that runs by default:
typeList = ["Normal", "Fire", "Water", "Grass", "Flying", "Fighting", "Poison", "Electric", "Ground", "Rock", "Psychic", "Ice", "Bug", "Ghost", "Steel", "Dragon", "Dark", "Fairy"]

# This is number of pokemon returned per type.
findThisMany = 5


In [ ]:
import pandas as pd
 
import pokemonMatch as match
 
runCases =  'basic'
 
whatFlag = runCases
 
#flag for running cases
#whatFlag = 'both' # 'basic' 'shadow' 'both'
 
# GamePress total CPO filename
gamepressFilenameShadow = 'shadow_comprehensive_dps_gamepress_L35_best_moves.csv'
gamepressFilename = 'basic_comprehensive_dps_gamepress_L35_all_moves.csv'
gamepressFilename = 'all_comprehensive_dps_gamepress_L35_all_moves.csv'
#gamepressFilename = 'mega_comprehensive_dps_gamepress_L35_all_moves.csv'
 
# setting the filters
typeFilter = []
topX = 0
 
# load data
allData = match.getGamepressData(gamepressFilename,typeFilter,topX)
    
# print(list(allData.columns))
 
# type lookup:
allData = match.typeAttackLookup(allData)
 
allData["Shadow"]=False
 
name = allData["Pokemon"]
nameloc = name.str.startswith("Shadow")
allData.loc[nameloc,"Shadow"]=True
 
# finding Pokémon with both attack types the same
#typeList = ["Normal", "Fire", "Water", "Grass", "Flying", "Fighting", "Poison", "Electric", "Ground", "Rock", "Psychic", "Ice", "Bug", "Ghost", "Steel", "Dragon", "Dark", "Fairy"]
 
setData = pd.DataFrame() 
 
#findThisMany = 5
 
# Setting empty strings for later
joinedStrings = ""
joinedStringsShad = ""
joinedStringsAll = ""
# setting a list of empty, for listing all the top x later
joinedIntBase = list()
joinedIntShad = list()
 
# big loop now:
for thisType in typeList:
    setDataTemp = match.findBothAttackOfType(allData,thisType)
    setData = pd.concat([setData,setDataTemp.head(findThisMany)],sort=False)
    topTypeSet = setDataTemp[["Pokemon","#","DPS^3*TDO","Fast move","Special move","TypeFast","Shadow"]]
    topTypeSetPrint = setDataTemp[["Pokemon","DPS^3*TDO","Fast move","Special move","TypeFast"]]
    topTypeSetPrint = topTypeSetPrint.iloc[:findThisMany]
    print(topTypeSetPrint.to_string(index=False))
    
    print(" ")
    
    # select top X
    topTypeSet = topTypeSet.head(findThisMany)
 
    # separate Shadow from non-Shadow
    shadTop = topTypeSet[topTypeSet["Shadow"]==True]
    shadTopNot = topTypeSet[topTypeSet["Shadow"]==False]
 
    #print(shadTopNot.head(5))
 
    #print(len(shadTopNot))
 
    if len(shadTopNot) > 0:
        # base
        shadString = ""
        typeListTemp = shadTopNot["#"].head(findThisMany)
        joinedIntBase.extend(typeListTemp)
        typeStr = [str(int(inte)) for inte in typeListTemp]
        typeStr = ",".join(typeStr)
        #print(typeStr)
        allString = "Type: "+thisType+" top "+str(findThisMany)+" - "+typeStr+shadString
    else:
        allString = "Type: "+thisType+" not found in base"
    print(allString) 
 
    if len(shadTop) > 0:
        # shadow
        shadString = "&shadow"
        typeListTemp = shadTop["#"].head(findThisMany)
        joinedIntShad.extend(typeListTemp)
        typeStr = [str(int(inte)) for inte in typeListTemp]
        typeStr = ",".join(typeStr)
        #print(typeStr)
        allStringShad = "Type: Shadow "+thisType+" top "+str(findThisMany)+" - "+typeStr+shadString
    else:
        allStringShad = "Type: "+thisType+" not found in shadow"
    print(allStringShad)  
 
    joinedStrings = joinedStrings + "\n" + allString
    joinedStringsShad = joinedStringsShad + "\n" + allStringShad
 
    joinedStringsAll = joinedStringsAll + "\n" + allString + "\n" + allStringShad
 
    print(" ")
 
print("------------------------------------")
print("This is a summary list of pokemon per types")
 
print(joinedStrings)
 
print(joinedStringsShad)
 
print(joinedStringsAll)
 
print("")
print("------------------------------------")
print("This is a summary list of all pokemon gathered together of all types")
 
# make the lists sorted and unique:
joinedIntBase = list(sorted(set(joinedIntBase)))
joinedIntShad = list(sorted(set(joinedIntShad)))
 
typeStr = [str(int(inte)) for inte in joinedIntBase]
typeStr = ",".join(typeStr)
print("List of combined top "+str(findThisMany)+" base pokemon of all types: ")
print(typeStr)
 
typeStr = [str(int(inte)) for inte in joinedIntShad]
typeStr = ",".join(typeStr)
print("List of combined top "+str(findThisMany)+" Shadow pokemon of all types: ")
print(typeStr+"&shadow")
 
 
#print(allData[allData["Pokemon"]=="Kyogre"])
 
#print(allData.head(20))

          Pokemon  DPS^3*TDO Fast move Special move TypeFast
 Shadow Porygon-Z     1781.3   Lock-On   Hyper Beam   Normal
        Porygon-Z     1255.1   Lock-On       Return   Normal
   Shadow Ambipom      807.2   Scratch   Hyper Beam   Normal
  Shadow Porygon2      769.7   Lock-On   Hyper Beam   Normal
          Ambipom      599.5   Scratch       Return   Normal
 
Type: Normal top 5 - 474,424
Type: Shadow Normal top 5 - 474,424,233&shadow
 
          Pokemon  DPS^3*TDO  Fast move  Special move TypeFast
     Shadow Entei     3031.7  Fire Fang      Overheat     Fire
         Reshiram     2943.1  Fire Fang      Overheat     Fire
   Shadow Moltres     2916.5  Fire Spin      Overheat     Fire
 Shadow Charizard     2421.3  Fire Spin    Blast Burn     Fire
  Shadow Arcanine     1824.6  Fire Fang  Flamethrower     Fire
 
Type: Fire top 5 - 643
Type: Shadow Fire top 5 - 244,146,6,59&shadow
 
         Pokemon  DPS^3*TDO  Fast move  Special move TypeFast
 Shadow Swampert     2747.5  Water Gun  H